<a href="https://colab.research.google.com/github/crunchdomo/llm_conversation/blob/main/test_base_openai_baselne.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# from google.colab import userdata
# userdata.get('OPENAI_API_KEY')

In [17]:
import openai
import json
from datetime import datetime
import re
import random
import torch
from transformers import AutoTokenizer, pipeline

# device = 0 if torch.cuda.is_available() else -1

class ModelHandler:
    def __init__(self, model_name):
        self.model_name = model_name
        self.device = 0 if torch.cuda.is_available() else -1

        if "llama" in model_name.lower():
            self._init_llama()
        else:
            self._init_openai()

    def _init_openai(self):
        self.client = openai.OpenAI(
            api_key="",  # Replace with your key
            base_url="https://api.deepinfra.com/v1/openai" if "grok" in self.model_name else None
        )

    def _init_llama(self):
        self.tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct")
        self.pipe = pipeline(
            task="text-generation",
            model=self.model_name,
            tokenizer=self.tokenizer,
            device=self.device,
            torch_dtype=torch.bfloat16
        )
        self.pipe.tokenizer.pad_token = self.pipe.tokenizer.eos_token

    def format_llama_prompt(self, messages):
        B_INST, E_INST = "<|begin_of_text|><|start_header_id|>user<|end_header_id|>", "<|eot_id|>"
        return "".join(
            f"{B_INST}{msg['content']}{E_INST}" if msg['role'] == 'user'
            else f"<|start_header_id|>assistant<|end_header_id|>{msg['content']}<|eot_id|>"
            for msg in messages
        )

    def generate(self, messages, max_tokens=512):
        if hasattr(self, 'client'):
            response = self.client.chat.completions.create(
                model=self.model_name,
                messages=messages,
                max_tokens=max_tokens,
                temperature=0.7
            )
            return response.choices[0].message.content
        else:
            prompt = self.format_llama_prompt(messages)
            outputs = self.pipe(prompt, max_new_tokens=max_tokens)
            return outputs[0]["generated_text"][len(prompt):].strip()

def parse_steps(instructions):
    """Parse instructions into individual steps"""
    steps = re.split(r'\n\s*\d+\.\s+', instructions.strip())
    return [f"{i+1}. {s.strip()}" for i, s in enumerate(steps) if s.strip()]



def save_conversation(chat_history, filename=None):
    if not filename:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"cooking_session_{timestamp}.json"

    with open(filename, 'w') as f:
        json.dump(chat_history, f, indent=2)
    print(f"Saved to {filename}")

# Recipe data structure
recipe = {
    "title": "Miso-Butter Roast Chicken With Acorn Squash Panzanella",
    "ingredients": [
        '1 (3½–4-lb.) whole chicken', '2¾ tsp. kosher salt, divided, plus more',
        '2 small acorn squash (about 3 lb. total)', '2 Tbsp. finely chopped sage',
        '1 Tbsp. finely chopped rosemary', '6 Tbsp. unsalted butter, melted, plus 3 Tbsp. room temperature',
        '¼ tsp. ground allspice', 'Pinch of crushed red pepper flakes', 'Freshly ground black pepper',
        '⅓ loaf good-quality sturdy white bread, torn into 1" pieces (about 2½ cups)',
        '2 medium apples (such as Gala or Pink Lady; about 14 oz. total), cored, cut into 1" pieces',
        '2 Tbsp. extra-virgin olive oil', '½ small red onion, thinly sliced',
        '3 Tbsp. apple cider vinegar', '1 Tbsp. white miso', '¼ cup all-purpose flour',
        '2 Tbsp. unsalted butter, room temperature', '¼ cup dry white wine',
        '2 cups unsalted chicken broth', '2 tsp. white miso', 'Kosher salt, freshly ground pepper'
    ],
    "instructions": """
        1. Pat chicken dry with paper towels, season all over with 2 tsp. salt, and tie legs together with kitchen twine. Let sit at room temperature 1 hour.

        2. Meanwhile, halve squash and scoop out seeds. Run a vegetable peeler along ridges of squash halves to remove skin. Cut each half into ½"-thick wedges; arrange on a rimmed baking sheet.

        3. Combine sage, rosemary, and 6 Tbsp. melted butter in a large bowl; pour half of mixture over squash on baking sheet. Sprinkle squash with allspice, red pepper flakes, and ½ tsp. salt and season with black pepper; toss to coat.

        4. Add bread, apples, oil, and ¼ tsp. salt to remaining herb butter in bowl; season with black pepper and toss to combine. Set aside.
        Place onion and vinegar in a small bowl; season with salt and toss to coat. Let sit, tossing occasionally, until ready to serve.

        5. Place a rack in middle and lower third of oven; preheat to 425°F. Mix miso and 3 Tbsp. room-temperature butter in a small bowl until smooth. Pat chicken dry with paper towels, then rub or brush all over with miso butter. Place chicken in a large cast-iron skillet and roast on middle rack until an instant-read thermometer inserted into the thickest part of breast registers 155°F, 50–60 minutes. (Temperature will climb to 165°F while chicken rests.) Let chicken rest in skillet at least 5 minutes, then transfer to a plate; reserve skillet.

        6. Meanwhile, roast squash on lower rack until mostly tender, about 25 minutes. Remove from oven and scatter reserved bread mixture over, spreading into as even a layer as you can manage. Return to oven and roast until bread is golden brown and crisp and apples are tender, about 15 minutes. Remove from oven, drain pickled onions, and toss to combine. Transfer to a serving dish.

        7. Using your fingers, mash flour and butter in a small bowl to combine.

        8. Set reserved skillet with chicken drippings over medium heat. You should have about ¼ cup, but a little over or under is all good. (If you have significantly more, drain off and set excess aside.) Add wine and cook, stirring often and scraping up any browned bits with a wooden spoon, until bits are loosened and wine is reduced by about half (you should be able to smell the wine), about 2 minutes. Add butter mixture; cook, stirring often, until a smooth paste forms, about 2 minutes. Add broth and any reserved drippings and cook, stirring constantly, until combined and thickened, 6–8 minutes. Remove from heat and stir in miso. Taste and season with salt and black pepper.

        9. Serve chicken with gravy and squash panzanella alongside.
    """
}

# # Automated inputs
# automated_inputs = [
#     "Got it! Ready for step 1.",
#     "Done Continue.",
#     "Done Continue.",
#     "Done Continue.",
#     "Done Continue.",
#     "Done Continue.",
#     "Done Continue.",
#     "Done Continue.",
#     "Done Continue.",
#     "Done Continue.",
#     "Done Continue.",
#     "exit"
# ]

def generate_llm_response(handler, role, chat, prompt, max_tokens=256):
    chat.append({"role": role, "content": prompt})
    response = handler.generate(chat, max_tokens=max_tokens)
    chat.append({"role": "assistant" if role == "user" else "user", "content": response})
    return response

def manager_decision(manager_handler, chat, step_index, steps, conversation_state):
    # The manager LLM decides the next action
    prompt = (
        f"You are the conversation manager for a cooking lesson. "
        f"Current step: {step_index+1}/{len(steps)}. "
        f"Conversation state: {conversation_state}. "
        "Decide what to do next: "
        "- Present next step, "
        "- End the conversation if all steps are done or student is satisfied. "
        "If the previous answer was clear and no clarification is needed, reply with 'CONFIRM'. Only reply with 'ASK_QUESTION' if there is genuine ambiguity, missing information, or a natural follow-up question a real user would ask."
    )

    response = manager_handler.generate(chat + [{"role": "system", "content": prompt}], max_tokens=32)
    return response.strip().upper()

def student_generate(student_handler, chat, step_content):
    # Student LLM responds to the step, possibly with a question
    prompt = (
        f"You are a cooking student. The chef just presented: '{step_content}'. "
        "Respond as a student: If the chef's explanation is clear, reply with a brief confirmation like 'OK' or 'Thanks, I understand.' Only ask a question if you truly need clarification or want to know more.."
    )
    return student_handler.generate(chat + [{"role": "system", "content": prompt}], max_tokens=128)

def parse_user_recipe(user_input):
    """
    Very basic parser:
    - If the user provides a recipe name, you could fetch from a database or prompt an LLM to generate it.
    - If the user pastes a full recipe, try to extract title, ingredients, and instructions.
    """
    # Check if user pasted a full recipe (very naive)
    if "ingredients" in user_input.lower() and "instructions" in user_input.lower():
        # Try to extract sections
        title_match = re.search(r"^(.*?)(?:\n|$)", user_input.strip())
        title = title_match.group(1).strip() if title_match else "Untitled Recipe"
        ingredients_match = re.search(r"ingredients\s*:(.*?)(instructions\s*:|$)", user_input, re.IGNORECASE | re.DOTALL)
        instructions_match = re.search(r"instructions\s*:(.*)", user_input, re.IGNORECASE | re.DOTALL)
        ingredients = [i.strip("-• \n") for i in ingredients_match.group(1).split("\n") if i.strip()] if ingredients_match else []
        instructions = instructions_match.group(1).strip() if instructions_match else ""
        return {"title": title, "ingredients": ingredients, "instructions": instructions}
    else:
        # Only recipe name provided
        return {"title": user_input.strip(), "ingredients": [], "instructions": ""}

def request_recipe_and_start(model_names):
    chef_handler = ModelHandler(model_names['chef'])
    chat = [{
        "role": "system",
        "content": (
            "You are a friendly master chef assistant. "
            "First, introduce yourself and ask the user what recipe they would like to cook today. "
            "Wait for the user to provide a recipe name or paste the full recipe. "
            "Once you have the recipe, guide the user step-by-step."
        )
    }]
    # Phase 1: Chef introduces itself and asks for a recipe
    chef_intro = chef_handler.generate(chat + [{"role": "user", "content": "Hello"}])
    print("\nChef:", chef_intro)
    chat.append({"role": "assistant", "content": chef_intro})

    # Phase 2: User provides a recipe (simulate input for now)
    user_recipe_input = input("\nPlease provide the recipe you'd like to cook (name or full recipe):\n")
    chat.append({"role": "user", "content": user_recipe_input})

    # Parse the recipe
    recipe_data = parse_user_recipe(user_recipe_input)
    if not recipe_data["instructions"]:
        # If only a recipe name is provided, ask the chef LLM to generate a recipe
        prompt = (
            f"Please provide a detailed recipe for '{recipe_data['title']}'. "
            "Include an ingredients list and step-by-step instructions."
        )
        recipe_text = chef_handler.generate(chat + [{"role": "user", "content": prompt}], max_tokens=512)
        # Try to parse the generated recipe
        recipe_data = parse_user_recipe(recipe_text)
        chat.append({"role": "assistant", "content": recipe_text})

    # Now begin the step-by-step cooking process
    cook_recipe_llm_managed(recipe_data, model_names, chat=chat)

def cook_recipe_llm_managed(recipe_data, model_names, chat=None):
    chef_handler = ModelHandler(model_names['chef'])
    student_handler = ModelHandler(model_names['student'])
    manager_handler = ModelHandler(model_names['manager'])

    steps = parse_steps(recipe_data['instructions'])
    if chat is None:
        chat = [{
            "role": "system",
            "content": (
                f"You are a master chef guiding through: {recipe_data['title']}\n"
                "- Begin each step with 'STEP: [NUMBER]'\n"
                "- Use metric measurements\n"
                "- Answer questions about specific steps using their numbers"
            )
        }]
    else:
        chat.append({
            "role": "system",
            "content": (
                f"You are a master chef guiding through: {recipe_data['title']}\n"
                "- Begin each step with 'STEP: [NUMBER]'\n"
                "- Use metric measurements\n"
                "- Answer questions about specific steps using their numbers"
            )
        })
    step_index = 0
    conversation_state = {
        "step_confirmed": False,
        "questions_asked": 0,
        "step": 1
    }

    while step_index < len(steps):
        # Present step if not yet confirmed
        if not conversation_state["step_confirmed"]:
            step_content = f"Present step {step_index+1} clearly: {steps[step_index]}"
            chef_response = chef_handler.generate(chat + [{"role": "user", "content": step_content}])
            chat.append({"role": "assistant", "content": chef_response})

        # Student always asks a question first (unless already done)
        if conversation_state["questions_asked"] == 0:
            student_prompt = (
                f"You are a curious cooking student. The chef just presented: '{chef_response}'. "
                "Ask a clarifying or substitution question about this step before confirming."
            )
            student_question = student_handler.generate(chat + [{"role": "system", "content": student_prompt}])
            chat.append({"role": "user", "content": student_question})

            # Chef answers
            chef_answer = chef_handler.generate(chat)
            chat.append({"role": "assistant", "content": chef_answer})
            conversation_state["questions_asked"] += 1

        # Student confirms step after question is answered
        student_confirm_prompt = (
            "If the chef's answer is clear, reply with a brief confirmation like 'OK' or 'Understood.' "
            "If not, ask a clarifying question."
        )

        student_confirm = student_handler.generate(chat + [{"role": "system", "content": student_confirm_prompt}])
        chat.append({"role": "user", "content": student_confirm})
        conversation_state["step_confirmed"] = True

        # Manager decides if we should proceed
        manager_prompt = (
            f"You are the conversation manager for a cooking lesson. "
            f"Step: {step_index+1}/{len(steps)}. "
            f"Conversation so far: {chat[-5:]} "
            "Should the student ask a question, or just confirm? Reply with 'ASK_QUESTION' or 'CONFIRM'."
        )
        manager_decision = manager_handler.generate(chat + [{"role": "system", "content": manager_prompt}], max_tokens=8)
        if "END" in manager_decision:
            chat.append({"role": "system", "content": "CONVO-COMPLETE"})
            break
        else:
            # Reset state for next step
            step_index += 1
            conversation_state = {
                "step_confirmed": False,
                "questions_asked": 0,
                "step": step_index + 1
            }
        print(f"\n[Manager Decision] Step {step_index+1}: {manager_decision}")
        print(f"[Manager Prompt] {manager_prompt}")

    save_conversation(chat, f"cooking_session_llm_managed.json")
    return chat

# Usage:
model_names = {
    "chef": "gpt-4-turbo",
    "student": "gpt-4-turbo",  # or another LLM
    "manager": "gpt-4-turbo"     # or another LLM
}
request_recipe_and_start(model_names)


Chef: Hello there! I'm your friendly master chef assistant. What recipe would you like to cook today? Feel free to tell me the name of the dish or paste the full recipe here, and I'll help guide you through it step-by-step!

Please provide the recipe you'd like to cook (name or full recipe):
    "title": "Miso-Butter Roast Chicken With Acorn Squash Panzanella",     "ingredients": [         '1 (3½–4-lb.) whole chicken', '2¾ tsp. kosher salt, divided, plus more',         '2 small acorn squash (about 3 lb. total)', '2 Tbsp. finely chopped sage',         '1 Tbsp. finely chopped rosemary', '6 Tbsp. unsalted butter, melted, plus 3 Tbsp. room temperature',         '¼ tsp. ground allspice', 'Pinch of crushed red pepper flakes', 'Freshly ground black pepper',         '⅓ loaf good-quality sturdy white bread, torn into 1" pieces (about 2½ cups)',         '2 medium apples (such as Gala or Pink Lady; about 14 oz. total), cored, cut into 1" pieces',         '2 Tbsp. extra-virgin olive oil', '½ smal